In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import cudf

In [2]:
df = cudf.read_parquet('../../data/train_data/test.parquet')
df = df.loc[df['type']!=0]

df['hour'] = df.ts % (60*60*24)
df['day'] = df.ts % (60*60*24*7)
print( df.shape )
df.head()

(725300, 6)


,session,aid,ts,type,hour,day
7,11098530,409236,1661120532,1,80532,339732
28,11098531,1728212,1661119746,2,79746,338946
29,11098531,452188,1661119746,2,79746,338946
30,11098531,1271998,1661119746,2,79746,338946
31,11098531,396199,1661119746,2,79746,338946


In [3]:
item_features = df.groupby('aid').agg({'type':'mean','aid':'count','session':'nunique','hour':'mean','day':'mean'})
item_features.columns = ['buy_ratio','count_item','count_user','hour_mean','day_mean']

In [4]:
item_features.head()

,buy_ratio,count_item,count_user,hour_mean,day_mean
aid,,,,,
3,1.068966,29,19,54715.241379,215598.000000
11,1.000000,1,1,43551.000000,561951.000000
21,1.000000,2,2,61765.500000,320965.500000
32,1.000000,1,1,72829.000000,159229.000000
43,1.000000,3,3,71587.333333,273187.333333


In [5]:
item_features2 = df.groupby('aid').agg({'type':'std','hour':'std','day':'std'}).fillna(-1)
item_features2.columns = ['buy_ratio_std','hour_std','day_std']

f32 = ['buy_ratio_std','hour_std','day_std']
for c in f32: item_features2[c] = item_features2[c].astype('float32')

In [6]:
item_features2.head()

,buy_ratio_std,hour_std,day_std
aid,,,
707464,-1.0,-1.000000,-1.000000
1604073,0.0,23982.941406,98205.109375
807853,0.0,151.320847,151.320847
1222453,-1.0,-1.000000,-1.000000
914505,-1.0,-1.000000,-1.000000


In [7]:
item_features['repeat'] = item_features.count_item / item_features.count_user

f32 = ['buy_ratio','hour_mean','day_mean','repeat']
for c in f32: item_features[c] = item_features[c].astype('float32')

i32 = ['count_item','count_user']
for c in i32: item_features[c] = item_features[c].astype('int32')

In [8]:
item_features = cudf.concat([item_features,item_features2],axis=1)

In [9]:
item_features.head()

,buy_ratio,count_item,count_user,hour_mean,day_mean,repeat,buy_ratio_std,hour_std,day_std
aid,,,,,,,,,
3,1.068966,29,19,54715.242188,215598.00000,1.526316,0.257881,16069.186523,122489.679688
11,1.000000,1,1,43551.000000,561951.00000,1.000000,-1.000000,-1.000000,-1.000000
21,1.000000,2,2,61765.500000,320965.50000,1.000000,0.000000,16122.742188,382686.906250
32,1.000000,1,1,72829.000000,159229.00000,1.000000,-1.000000,-1.000000,-1.000000
43,1.000000,3,3,71587.335938,273187.34375,1.000000,0.000000,9207.145508,129540.929688


In [10]:
item_features.dtypes

buy_ratio        float32
count_item         int32
count_user         int32
hour_mean        float32
day_mean         float32
repeat           float32
buy_ratio_std    float32
hour_std         float32
day_std          float32
dtype: object

In [11]:
COLS = [f'{x}12' for x in item_features.columns]
item_features.columns = COLS
print(COLS)

['buy_ratio12', 'count_item12', 'count_user12', 'hour_mean12', 'day_mean12', 'repeat12', 'buy_ratio_std12', 'hour_std12', 'day_std12']


In [12]:
item_features.to_parquet('../../data/item_user_features/item41.pqt')